In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator    
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16  # Import the VGG16 model
from glob import glob

train_path = './tranning12'
validation_path = './testing21'

# no. of classes in recognition data
className = glob(train_path + "/*")
NumberofClass = len(className)
print("NumberofClass:", NumberofClass)

train_datagen = ImageDataGenerator(rescale=1/255,
                  shear_range=0.3,
                  horizontal_flip=True,
                  vertical_flip=True,  # Added vertical flip
                  rotation_range=20,   # Added rotation
                  zoom_range=0.3
                  )
val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                train_path,
                target_size=(224,224),
                batch_size=15,  # Reduced batch size for better generalization
                color_mode="rgb",
                class_mode="categorical"
                )
val_generator = val_datagen.flow_from_directory(
                validation_path,
                target_size=(224,224),
                batch_size=15,  # Reduced batch size for better generalization
                color_mode="rgb",
                class_mode="categorical"
                )

# Load the VGG16 model, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=tf.keras.Input(shape=(224, 224, 3)))

# Construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(NumberofClass, activation="softmax")(headModel)

# Place the head FC model on top of the base model (this will become the actual model we will train)
model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)

# Loop over all layers in the base model and freeze them so they will not be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# Compile with adaptive learning rate
opt = tf.keras.optimizers.Adam(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9))
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
callback = EarlyStopping(monitor='val_loss', patience=4)

# Fit the model
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=30,
                    callbacks=[callback])  # Added early stopping

#plot for fake vs real classification
# Plot for accuracy vs validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], 'go--', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'ro--', label='Validation Accuracy')
plt.title('Training Accuracy vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], 'go--', label='Training Loss')
plt.plot(history.history['val_loss'], 'ro--', label='Validation Loss')
plt.title('Training Loss vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
# Assuming you have already trained the model and obtained predictions on the validation set
predictions = model.predict(val_generator)

# Convert one-hot encoded labels back to categorical labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = val_generator.classes

# Get class labels from the generator
class_labels = list(val_generator.class_indices.keys())

# Print classification report
print(classification_report(true_labels, predicted_labels, target_names=class_labels))


In [ ]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = './dataset/tranning/real/3asd.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(224, 224))

# Preprocess the image
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

# Predict the image
img_preprocessed = tf.keras.applications.vgg16.preprocess_input(img_batch)
prediction = model.predict(img_preprocessed)

# Get the class labels from the training generator
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}

# Get the class label of the prediction
predicted_class = class_labels[np.argmax(prediction)]

# Display the image
plt.imshow(img)
plt.axis('off')

# Display the prediction
plt.title(f"Predicted: {predicted_class}")
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = './testing/fake/images.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(224, 224))

# Preprocess the image
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

# Predict the image
img_preprocessed = tf.keras.applications.vgg16.preprocess_input(img_batch)
prediction = model.predict(img_preprocessed)

# Get the class labels from the training generator
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}

# Get the class label of the prediction
predicted_class = class_labels[np.argmax(prediction)]

# Display the image
plt.imshow(img)
plt.axis('off')

# Display the prediction
plt.title(f"Predicted: {predicted_class}")
plt.show()